In [3]:
# تثبيت مكتبة Kaggle
!pip install kaggle

import os

# قم بتعيين متغيرات البيئة باستخدام الأسرار التي أضفتها
os.environ['KAGGLE_USERNAME'] = 'YOUR_KAGGLE_USERNAME' # استبدل باسم المستخدم الخاص بك
os.environ['KAGGLE_KEY'] = 'YOUR_KAGGLE_KEY' # استبدل بمفتاح الـ API الخاص بك

# ابحث عن مجموعة بيانات كبيرة وحملها (كمثال)
# لاستخدام مجموعة بيانات أخرى، غيّر الأمر التالي
!kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store

# فك ضغط الملف
!unzip ecommerce-behavior-data-from-multi-category-store.zip

Dataset URL: https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store
License(s): copyright-authors
 99%|██████████████████████████████████████▊| 4.27G/4.29G [00:05<00:00, 409MB/s]
100%|███████████████████████████████████████| 4.29G/4.29G [00:05<00:00, 849MB/s]
Archive:  ecommerce-behavior-data-from-multi-category-store.zip
  inflating: 2019-Nov.csv            
  inflating: 2019-Oct.csv            


In [5]:
import pandas as pd
import time

# تحديد اسم الملف
file_name = '2019-Oct.csv'

# قياس الوقت والمساحة
start_time = time.time()

# إنشاء مكرر (iterator) لقراءة الملف في أجزاء
chunk_iter = pd.read_csv(file_name, chunksize=100000) # قراءة 100 ألف صف في كل مرة

# معالجة كل جزء على حدة
# في هذا المثال، سنقوم فقط بحساب عدد الصفوف
total_rows = 0
for chunk in chunk_iter:
    total_rows += len(chunk)
    # يمكنك إجراء عمليات أخرى هنا على كل جزء
    # مثل: chunk['price'].mean()

end_time = time.time()

print(f"Total rows: {total_rows}")
print(f"Time taken with chunks: {end_time - start_time:.2f} seconds")

# ملاحظة: استهلاك الذاكرة هنا يكون فقط لحجم الجزء الواحد، وليس الملف كاملاً.

Total rows: 42448764
Time taken with chunks: 86.53 seconds


In [6]:
import dask.dataframe as dd
import time

# تحديد اسم الملف
file_name = '2019-Oct.csv'

# قياس الوقت
start_time = time.time()

# قراءة الملف باستخدام Dask (القراءة هنا كسولة "lazy"، أي لا يتم تحميل البيانات فعلياً)
dask_df = dd.read_csv(file_name)

# لتنفيذ عملية حسابية، يجب استدعاء compute()
# هنا سنحسب عدد الصفوف
total_rows = len(dask_df)

end_time = time.time()

print(f"Total rows: {total_rows}")
print(f"Time taken with Dask: {end_time - start_time:.2f} seconds")

# ملاحظة: Dask لا تقوم بتحميل البيانات إلا عند تنفيذ أمر يتطلب نتيجة مثل .compute() أو len()

Total rows: 42448764
Time taken with Dask: 58.10 seconds


In [10]:
import os

# تحديد مسارات الملفات التي تريد حذفها
zip_file_path = '/kaggle/working/ecommerce-behavior-data-from-multi-category-store.zip'
oct_csv_path = '/kaggle/working/2019-Oct.csv'

# حذف الملفات لتفريغ مساحة على القرص
if os.path.exists(zip_file_path):
    os.remove(zip_file_path)
    print(f"تم حذف: {zip_file_path}")

if os.path.exists(oct_csv_path):
    os.remove(oct_csv_path)
    print(f"تم حذف: {oct_csv_path}")

print("\nاكتمل التنظيف. لديك الآن مساحة أكبر على القرص. ✅")

تم حذف: /kaggle/working/ecommerce-behavior-data-from-multi-category-store.zip
تم حذف: /kaggle/working/2019-Oct.csv

اكتمل التنظيف. لديك الآن مساحة أكبر على القرص. ✅


In [12]:
# ----------------------------------------------------
# الطريقة الثالثة: التحويل إلى Parquet للقراءة السريعة (النسخة المصححة)
# ----------------------------------------------------
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import time # Make sure time is imported

parquet_path = '/kaggle/working/data.parquet'
file_path = '/kaggle/working/2019-Nov.csv'

# --- 1. عملية التحويل (تُنفذ مرة واحدة فقط) ---
print(f"\nبدء تحويل الملف {file_path} إلى صيغة Parquet...")
start_time = time.time()

# نستخدم chunksize لقراءة ملف CSV الضخم
chunk_iterator = pd.read_csv(file_path, chunksize=500_000)

# نأخذ أول قطعة لتحديد الهيكل (schema) للملف
first_chunk = next(chunk_iterator)
schema = pa.Table.from_pandas(first_chunk).schema

# نفتح ملف Parquet للكتابة باستخدام ParquetWriter
# NOTICE the corrected indentation below
with pq.ParquetWriter(parquet_path, schema=schema, compression='snappy') as writer:
    # نكتب القطعة الأولى التي قرأناها بالفعل
    writer.write_table(pa.Table.from_pandas(first_chunk))
    
    # نكمل كتابة باقي القطع في الملف
    for chunk in chunk_iterator:
        writer.write_table(pa.Table.from_pandas(chunk))

end_time = time.time()
print(f"اكتمل التحويل إلى Parquet في {end_time - start_time:.2f} ثانية.")


# --- 2. قراءة ملف Parquet السريعة (هذا الجزء يبقى كما هو) ---
print("\nالآن، قراءة ملف Parquet المحول...")
start_time_read = time.time()

df_from_parquet = pd.read_parquet(parquet_path)

end_time_read = time.time()
print(f"تمت قراءة ملف Parquet بالكامل في {end_time_read - start_time_read:.2f} ثانية فقط!")

# يمكنك الآن تحليل البيانات بسرعة
print(f"متوسط السعر من ملف Parquet: {df_from_parquet['price'].mean():.2f}")


بدء تحويل الملف /kaggle/working/2019-Nov.csv إلى صيغة Parquet...
اكتمل التحويل إلى Parquet في 209.52 ثانية.

الآن، قراءة ملف Parquet المحول...
تمت قراءة ملف Parquet بالكامل في 41.24 ثانية فقط!
متوسط السعر من ملف Parquet: 292.46
